In [22]:
import os
import numpy as np
import cv2
import argparse
import time

In [23]:
from tqdm import tqdm

In [4]:
#for renaming
#for filename in os.listdir():
#    file =filename.split(".")
#    if(file[-1]=="txt"):
#        new_list=[]
#        #print(filename)
#        with open(filename, "r+") as f:
#            old = f.read()
#            old=old.split("\n")
#            for i in old:
#                i=i.split(" ")
#                i[0]='2'
#                new_list.append(" ".join(i))
#        with open(filename, "w") as f:
#            f.write("\n".join(new_list))

# Data Augmentation

In [24]:
#convert from Yolo_mark to opencv format
def yoloFormattocv(x1, y1, x2, y2, H, W):
    bbox_width = x2 * W
    bbox_height = y2 * H
    center_x = x1 * W
    center_y = y1 * H

    voc = []

    voc.append(center_x - (bbox_width / 2))
    voc.append(center_y - (bbox_height / 2))
    voc.append(center_x + (bbox_width / 2))
    voc.append(center_y + (bbox_height / 2))

    return [int(v) for v in voc]

In [25]:
# convert from opencv format to yolo format
# H,W is the image height and width
def cvFormattoYolo(corner, H, W):
    bbox_W = corner[3] - corner[1]
    bbox_H = corner[4] - corner[2]

    center_bbox_x = (corner[1] + corner[3]) / 2
    center_bbox_y = (corner[2] + corner[4]) / 2

    return corner[0], round(center_bbox_x / W, 6), round(center_bbox_y / H, 6), round(bbox_W / W, 6), round(bbox_H / H, 6)

In [26]:
class yoloRotatebbox:
    def __init__(self, filename, image_ext, angle):
        assert os.path.isfile(filename + image_ext)
        assert os.path.isfile(filename + '.txt')
        

        self.filename = filename
        self.image_ext = image_ext
        self.angle = angle
        
        # Read image using cv2
        self.image = cv2.imread(self.filename + self.image_ext, 1)
        
        rotation_angle = self.angle * np.pi / 180
        self.rot_matrix = np.array(
            [[np.cos(rotation_angle), -np.sin(rotation_angle)], [np.sin(rotation_angle), np.cos(rotation_angle)]])

    def rotateYolobbox(self):

        new_height, new_width = self.rotate_image().shape[:2]

        f = open(self.filename + '.txt', 'r')

        f1 = f.readlines()

        new_bbox = []

        H, W = self.image.shape[:2]

        for x in f1:
            bbox = x.strip('\n').split(' ')
            if len(bbox) > 1:
                (center_x, center_y, bbox_width, bbox_height) = yoloFormattocv(float(bbox[1]), float(bbox[2]),
                                                                               float(bbox[3]), float(bbox[4]), H, W)

                upper_left_corner_shift = (center_x - W / 2, -H / 2 + center_y)
                upper_right_corner_shift = (bbox_width - W / 2, -H / 2 + center_y)
                lower_left_corner_shift = (center_x - W / 2, -H / 2 + bbox_height)
                lower_right_corner_shift = (bbox_width - W / 2, -H / 2 + bbox_height)

                new_lower_right_corner = [-1, -1]
                new_upper_left_corner = []

                for i in (upper_left_corner_shift, upper_right_corner_shift, lower_left_corner_shift,
                          lower_right_corner_shift):
                    new_coords = np.matmul(self.rot_matrix, np.array((i[0], -i[1])))
                    x_prime, y_prime = new_width / 2 + new_coords[0], new_height / 2 - new_coords[1]
                    if new_lower_right_corner[0] < x_prime:
                        new_lower_right_corner[0] = x_prime
                    if new_lower_right_corner[1] < y_prime:
                        new_lower_right_corner[1] = y_prime

                    if len(new_upper_left_corner) > 0:
                        if new_upper_left_corner[0] > x_prime:
                            new_upper_left_corner[0] = x_prime
                        if new_upper_left_corner[1] > y_prime:
                            new_upper_left_corner[1] = y_prime
                    else:
                        new_upper_left_corner.append(x_prime)
                        new_upper_left_corner.append(y_prime)
                #             print(x_prime, y_prime)

                new_bbox.append([bbox[0], new_upper_left_corner[0], new_upper_left_corner[1],
                                 new_lower_right_corner[0], new_lower_right_corner[1]])
        return new_bbox

    def rotate_image(self):
        """
        Rotates an image (angle in degrees) and expands image to avoid cropping
        """
        height, width = self.image.shape[:2]  # image shape has 3 dimensions
        image_center = (width / 2,
                        height / 2)  # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

        rotation_mat = cv2.getRotationMatrix2D(image_center, self.angle, 1.)

        # rotation calculates the cos and sin, taking absolutes of those.
        abs_cos = abs(rotation_mat[0, 0])
        abs_sin = abs(rotation_mat[0, 1])

        # find the new width and height bounds
        bound_w = int(height * abs_sin + width * abs_cos)
        bound_h = int(height * abs_cos + width * abs_sin)

        # subtract old image center (bringing image back to origin) and adding the new image center coordinates
        rotation_mat[0, 2] += bound_w / 2 - image_center[0]
        rotation_mat[1, 2] += bound_h / 2 - image_center[1]

        # rotate image with the new bounds and translated rotation matrix
        rotated_mat = cv2.warpAffine(self.image, rotation_mat, (bound_w, bound_h))
        return rotated_mat

In [ ]:
if __name__ == "__main__":
    angels=[45,70,90,115,135,155,180,205,225,250,270,295,315]
    for filename in tqdm(os.listdir()): 
        file =filename.split(".")
        if(file[-1]=="jpg"):
            image_name=file[0]
            image_ext="."+file[1]
        else:
            continue
        for angle in angels:
            im = yoloRotatebbox(image_name, image_ext, angle)
            bbox = im.rotateYolobbox()
            image = im.rotate_image()

            # to write rotateed image to disk
            cv2.imwrite(image_name+'_' + str(angle) + '.jpg', image)

            file_name = image_name+'_' + str(angle) + '.txt'
            #print("For angle "+str(angle))
            if os.path.exists(file_name):
                os.remove(file_name)

            # to write the new rotated bboxes to file
            for i in bbox:
                with open(file_name, 'a') as fout:
                    fout.writelines(
                        ' '.join(map(str, cvFormattoYolo(i, im.rotate_image().shape[0], im.rotate_image().shape[1]))) + '\n')

In [27]:
#for imges only
os.chdir("D:\\My ML Projects\\lenskart task\\sub-train\\Non-Power Reading\\Wayfarer")
if __name__ == "__main__":
    #angels=[45,55,90,115,135,155,180,205,225,250,270,295,315,345]
    angels=[90,180,275]
    for filename in tqdm(os.listdir()): 
        file =filename.split(".")
        if(file[-1]=="jpg"):
            image_name=file[0]
            image_ext="."+file[1]
        else:
            continue
        for angle in angels:
            im = yoloRotatebbox(image_name, image_ext, angle)
            image = im.rotate_image()

            # to write rotateed image to disk
            cv2.imwrite(image_name+'_' + str(angle) + '.jpg', image)

100%|████████████████████████████████████████████████████████████████████████████████| 279/279 [00:14<00:00, 18.83it/s]


In [32]:
from skimage.transform import rotate
from skimage.util import random_noise, img_as_ubyte
from skimage.io import imread
import cv2, random

In [29]:
def h_flip(image):
    return np.fliplr(image)

def v_flip(image):
    return np.flipud(image)

def add_noise(image):
    return random_noise(image)

def blur_image(image):
    return cv2.GaussianBlur(image,(9,9),0)

In [30]:
transformations={"horizontal flip":h_flip,
                 "vertical flip": v_flip,
                 "adding noise":add_noise,
                 "making blur":blur_image
}

In [35]:
def Augment_images_gerenator():
    image_path="D:/My ML Projects/lenskart task/sub-train/eyeframe/Wayfarer"
    augmentated_path="D:/My ML Projects/lenskart task/sub-train/eyeframe/Wayfarer"
    images=[]
    for im in os.listdir(image_path):
        images.append(os.path.join(image_path,im))
    images_to_generate=1100
    i=1
    for i in tqdm(range(images_to_generate)):
        image=random.choice(images)
        original_image=imread(image)
        transformed_image=None
        n=0
        transformation_count=random.randint(1,len(transformations))

        while n<=transformation_count:
            key=random.choice(list(transformations))
            transformed_image=transformations[key](original_image)
            n+=1
        new_image_path="%s/aug_%s.jpg"%(augmentated_path,i)
        transformed_image=img_as_ubyte(transformed_image)
        transformed_image=cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(new_image_path, transformed_image)
        #i+=1
Augment_images_gerenator()

100%|██████████████████████████████████████████████████████████████████████████████| 1100/1100 [05:17<00:00,  3.47it/s]


In [36]:
def Augment_images_gerenator():
    image_path="D:/My ML Projects/lenskart task/sub-train/Non-Power Reading/Aviator"
    augmentated_path="D:/My ML Projects/lenskart task/sub-train/Non-Power Reading/Aviator"
    images=[]
    for im in os.listdir(image_path):
        images.append(os.path.join(image_path,im))
    images_to_generate=1000
    i=1
    for i in tqdm(range(images_to_generate)):
        image=random.choice(images)
        original_image=imread(image)
        transformed_image=None
        n=0
        transformation_count=random.randint(1,len(transformations))

        while n<=transformation_count:
            key=random.choice(list(transformations))
            transformed_image=transformations[key](original_image)
            n+=1
        new_image_path="%s/aug_%s.jpg"%(augmentated_path,i)
        transformed_image=img_as_ubyte(transformed_image)
        transformed_image=cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(new_image_path, transformed_image)
        #i+=1
Augment_images_gerenator()

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:35<00:00,  4.65it/s]


In [37]:
def Augment_images_gerenator():
    image_path="D:/My ML Projects/lenskart task/sub-train/Non-Power Reading/Oval"
    augmentated_path="D:/My ML Projects/lenskart task/sub-train/Non-Power Reading/Oval"
    images=[]
    for im in os.listdir(image_path):
        images.append(os.path.join(image_path,im))
    images_to_generate=1100
    i=1
    for i in tqdm(range(images_to_generate)):
        image=random.choice(images)
        original_image=imread(image)
        transformed_image=None
        n=0
        transformation_count=random.randint(1,len(transformations))

        while n<=transformation_count:
            key=random.choice(list(transformations))
            transformed_image=transformations[key](original_image)
            n+=1
        new_image_path="%s/aug_%s.jpg"%(augmentated_path,i)
        transformed_image=img_as_ubyte(transformed_image)
        transformed_image=cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(new_image_path, transformed_image)
        #i+=1
Augment_images_gerenator()

100%|██████████████████████████████████████████████████████████████████████████████| 1100/1100 [05:18<00:00,  3.46it/s]


In [38]:
def Augment_images_gerenator():
    image_path="D:/My ML Projects/lenskart task/sub-train/Non-Power Reading/Wayfarer"
    augmentated_path="D:/My ML Projects/lenskart task/sub-train/Non-Power Reading/Wayfarer"
    images=[]
    for im in os.listdir(image_path):
        images.append(os.path.join(image_path,im))
    images_to_generate=200
    i=1
    for i in tqdm(range(images_to_generate)):
        image=random.choice(images)
        original_image=imread(image)
        transformed_image=None
        n=0
        transformation_count=random.randint(1,len(transformations))

        while n<=transformation_count:
            key=random.choice(list(transformations))
            transformed_image=transformations[key](original_image)
            n+=1
        new_image_path="%s/aug_%s.jpg"%(augmentated_path,i)
        transformed_image=img_as_ubyte(transformed_image)
        transformed_image=cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(new_image_path, transformed_image)
        #i+=1
Augment_images_gerenator()

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:22<00:00,  8.90it/s]


In [ ]:
def Augment_images_gerenator():
    image_path="D:/My ML Projects/lenskart task/sub-train/eyeframe/Wayfarer"
    augmentated_path="D:/My ML Projects/lenskart task/sub-train/eyeframe/Wayfarer"
    images=[]
    for im in os.listdir(image_path):
        images.append(os.path.join(image_path,im))
    images_to_generate=1100
    i=1
    for i in tqdm(range(images_to_generate)):
        image=random.choice(images)
        original_image=imread(image)
        transformed_image=None
        n=0
        transformation_count=random.randint(1,len(transformations))

        while n<=transformation_count:
            key=random.choice(list(transformations))
            transformed_image=transformations[key](original_image)
            n+=1
        new_image_path="%s/aug_%s.jpg"%(augmentated_path,i)
        transformed_image=img_as_ubyte(transformed_image)
        transformed_image=cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(new_image_path, transformed_image)
        #i+=1
Augment_images_gerenator()

In [39]:
def Augment_images_gerenator():
    image_path="D:/My ML Projects/lenskart task/sub-train/sunglasses/Rectangle"
    augmentated_path="D:/My ML Projects/lenskart task/sub-train/sunglasses/Rectangle"
    images=[]
    for im in os.listdir(image_path):
        images.append(os.path.join(image_path,im))
    images_to_generate=300
    i=1
    for i in tqdm(range(images_to_generate)):
        image=random.choice(images)
        original_image=imread(image)
        transformed_image=None
        n=0
        transformation_count=random.randint(1,len(transformations))

        while n<=transformation_count:
            key=random.choice(list(transformations))
            transformed_image=transformations[key](original_image)
            n+=1
        new_image_path="%s/aug_%s.jpg"%(augmentated_path,i)
        transformed_image=img_as_ubyte(transformed_image)
        transformed_image=cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(new_image_path, transformed_image)
        #i+=1
Augment_images_gerenator()

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:57<00:00,  5.20it/s]
